<a href="https://colab.research.google.com/github/Arunavaonly/Image-Based-Clickbait-Detection-Model-Pipeline/blob/main/thumbnail_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import requests
from PIL import Image
import csv

dataset = pd.read_excel("BaitBuster-Bangla.xlsx", index_col= False)

final_dataset = dataset[['thumbnail','human_labeled']]
final_dataset = final_dataset.dropna()
final_dataset = final_dataset.sample(frac =1).reset_index(drop=True)
print(final_dataset.value_counts('human_labeled'))
print(final_dataset.head())
print(final_dataset.tail())


# Directory to store downloaded images
os.makedirs("data_dir/train/clickbait", exist_ok=True)
os.makedirs("data_dir/train/non-clickbait", exist_ok =True)

def download_image(url, save_path):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(save_path, "wb") as f:
                f.write(response.content)
    except Exception as e:
        print(f"Error downloading {url}: {e}")

# Example: Iterate over dataset and download images
for index, row in final_dataset.iterrows():  # Assuming dataset is a DataFrame
    thumbnail_url = row["thumbnail"]
    label = row["human_labeled"]
    if label == 'Clickbait':
        save_path = f"data_dir/train/clickbait/{index}.jpg"
        download_image(thumbnail_url, save_path)
    else:
        save_path = f"data_dir/train/non-clickbait/{index}.jpg"
        download_image(thumbnail_url, save_path)



human_labeled
Clickbait        5644
Not Clickbait    4356
Name: count, dtype: int64
                                        thumbnail  human_labeled
0  https://i.ytimg.com/vi/g1mJZh0R2IU/default.jpg  Not Clickbait
1  https://i.ytimg.com/vi/ZtzaXJ_kUVw/default.jpg      Clickbait
2  https://i.ytimg.com/vi/Fw6gnlE6F3w/default.jpg  Not Clickbait
3  https://i.ytimg.com/vi/mZ-DrisVSM4/default.jpg      Clickbait
4  https://i.ytimg.com/vi/vZbFpMxZ_gQ/default.jpg      Clickbait
                                           thumbnail  human_labeled
9995  https://i.ytimg.com/vi/cJOvWTr7fcY/default.jpg      Clickbait
9996  https://i.ytimg.com/vi/5IQDNB_P4k0/default.jpg      Clickbait
9997  https://i.ytimg.com/vi/AN4RkfchXoM/default.jpg      Clickbait
9998  https://i.ytimg.com/vi/hGGaREktoOU/default.jpg  Not Clickbait
9999  https://i.ytimg.com/vi/VEsrmownUYo/default.jpg  Not Clickbait


In [ ]:
lst = os.listdir("data_dir/train/clickbait")
lst2 = os.listdir("data_dir/train/non-clickbait")
print(len(lst))
print(len(lst2))

3963
4160


In [ ]:
os.makedirs("data_dir/val/clickbait", exist_ok=True)
os.makedirs("data_dir/val/non-clickbait", exist_ok =True)

In [ ]:
import os
import shutil

# Define paths
clickbait_dir = "data_dir/train/clickbait"
non_clickbait_dir = "data_dir/train/non-clickbait"
val_clickbait_dir = "data_dir/val/clickbait"
val_non_clickbait_dir = "data_dir/val/non-clickbait"

# Get lists of files
clickbait_files = os.listdir(clickbait_dir)
non_clickbait_files = os.listdir(non_clickbait_dir)

# Select 800 images from each category
clickbait_to_move = clickbait_files[:800]
non_clickbait_to_move = non_clickbait_files[:800]

# Move files for validation
for file in clickbait_to_move:
    shutil.move(os.path.join(clickbait_dir, file), os.path.join(val_clickbait_dir, file))

for file in non_clickbait_to_move:
    shutil.move(os.path.join(non_clickbait_dir, file), os.path.join(val_non_clickbait_dir, file))

print("Successfully moved 800 images to validation folders.")


Successfully moved 800 images to validation folders.


In [ ]:
val_list1 = os.listdir("data_dir/val/clickbait")
val_list2 = os.listdir("data_dir/val/non-clickbait")
print(len(val_list1))
print(len(val_list2))

800
800


In [ ]:
data = "/content/data_dir"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import time
import os

def train_model(model, dataloaders, criterion, optimizer, num_epochs=10, device="cuda"):
    since = time.time()

    # Keep track of best model weights
    best_model_wts = model.state_dict()
    best_acc = 0.0

    # Loop over each epoch
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        # Each epoch has a training and validation phase
        for phase in ["train", "val"]:
            if phase == "train":
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluation mode

            running_loss = 0.0
            running_corrects = 0
            total_samples = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)  # Get predicted class
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                total_samples += inputs.size(0)

            epoch_loss = running_loss / total_samples
            epoch_acc = running_corrects.double() / total_samples

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            # Deep copy the model if it's the best so far
            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    # Load best model weights
    model.load_state_dict(best_model_wts)
    time_elapsed = time.time() - since
    print(f"Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
    print(f"Best val Accuracy: {best_acc:.4f}")

    return model


In [ ]:
# Define data transforms
data_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Load data
data_dir = data
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ["train", "val"]}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=4)
               for x in ["train", "val"]}

# Define the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)

# Modify the final fully connected layer for binary classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: clickbait and non-clickbait
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
model = train_model(model, dataloaders, criterion, optimizer, num_epochs=10, device=device)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNe

Epoch 1/10
----------
train Loss: 0.1443 Acc: 0.9601
val Loss: 0.2549 Acc: 0.8831
Epoch 2/10
----------
train Loss: 0.1088 Acc: 0.9689
val Loss: 0.0950 Acc: 0.9688
Epoch 3/10
----------
train Loss: 0.0927 Acc: 0.9733
val Loss: 0.0983 Acc: 0.9706
Epoch 4/10
----------
train Loss: 0.0967 Acc: 0.9700
val Loss: 0.0905 Acc: 0.9713
Epoch 5/10
----------
train Loss: 0.0882 Acc: 0.9738
val Loss: 0.0858 Acc: 0.9744
Epoch 6/10
----------
train Loss: 0.0819 Acc: 0.9749
val Loss: 0.1282 Acc: 0.9656
Epoch 7/10
----------
train Loss: 0.0791 Acc: 0.9759
val Loss: 0.0868 Acc: 0.9756
Epoch 8/10
----------
train Loss: 0.0836 Acc: 0.9742
val Loss: 0.0909 Acc: 0.9719
Epoch 9/10
----------
train Loss: 0.0784 Acc: 0.9769
val Loss: 0.0916 Acc: 0.9700
Epoch 10/10
----------
train Loss: 0.0709 Acc: 0.9776
val Loss: 0.0987 Acc: 0.9750
Training complete in 12m 43s
Best val Accuracy: 0.9756


In [ ]:
torch.save(model.state_dict(), "resnet50_clickbait_classifier.pth")
print("Model training complete and saved!")


Model training complete and saved!


In [ ]:
from PIL import Image
import torch

# Get class names from the training dataset
class_names = image_datasets["train"].classes  # Extract class labels from the training dataset

# Load the saved model
model.load_state_dict(torch.load("resnet50_clickbait_classifier.pth"))
model.eval()

# Transform the input image and predict
def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")  # Ensure the image is in RGB format
    input_tensor = data_transforms["val"](image).unsqueeze(0).to(device)
    outputs = model(input_tensor)
    _, preds = torch.max(outputs, 1)  # Get the index of the predicted class
    return class_names[preds[0]]  # Map the predicted index to the class name

# Test on a new image
print(predict_image("default3.jpg"))


<ipython-input-10-7a163a647608>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("resnet50_clickbait_classifier.pth"))


clickbait


In [ ]:
from transformers import PreTrainedModel, PretrainedConfig
import torch.nn as nn
import torch

# Define a custom configuration class
class ResNetConfig(PretrainedConfig):
    model_type = "resnet"

    def __init__(self, num_labels=2, **kwargs):
        super().__init__(**kwargs)
        self.num_labels = num_labels


# Define a custom ResNet model class
class ResNetForImageClassification(PreTrainedModel):
    config_class = ResNetConfig

    def __init__(self, config):
        super().__init__(config)
        from torchvision import models
        self.resnet = models.resnet50(pretrained=False)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, config.num_labels)  # Modify output layer

    def forward(self, pixel_values, labels=None):
        logits = self.resnet(pixel_values)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
        return {"loss": loss, "logits": logits}


In [ ]:
from transformers import AutoProcessor

# Load the configuration and model
config = ResNetConfig(num_labels=2)
model = ResNetForImageClassification(config)

# Load the original state_dict
state_dict = torch.load("resnet50_clickbait_classifier.pth")

# Modify keys to match the expected prefix
new_state_dict = {f"resnet.{key}": value for key, value in state_dict.items()}

# Load the modified state_dict into the model
model.load_state_dict(new_state_dict)

# Save the model and processor
model.save_pretrained("resnet50_clickbait")
processor = AutoProcessor.from_pretrained("microsoft/resnet-50")
processor.save_pretrained("resnet50_clickbait")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-12-68a850aac82a>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

['resnet50_clickbait/preprocessor_config.json']

In [ ]:
!pip install huggingface_hub

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
The token `resnet50-clickbait-2` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authen

In [ ]:
from huggingface_hub import login
login()


In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.upload_folder(
    folder_path="resnet50_clickbait",  # Path to your folder containing model.safetensors, config.json, and preprocessor_config.json
    repo_id="Arunavaonly/YouTube-Clickbait-Video-Detection-Model",  # Replace with your model's repo ID
    repo_type="model"
)


model.safetensors:   0%|          | 0.00/94.3M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Arunavaonly/YouTube-Clickbait-Video-Detection-Model/commit/519a993f0408c8a10d2dfcc5d6f6cf403820fb3a', commit_message='Upload folder using huggingface_hub', commit_description='', oid='519a993f0408c8a10d2dfcc5d6f6cf403820fb3a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Arunavaonly/YouTube-Clickbait-Video-Detection-Model', endpoint='https://huggingface.co', repo_type='model', repo_id='Arunavaonly/YouTube-Clickbait-Video-Detection-Model'), pr_revision=None, pr_num=None)

In [ ]:
model.push_to_hub("Arunavaonly/YouTube-Clickbait-Video-Detection-Model")

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Arunavaonly/YouTube-Clickbait-Video-Detection-Model/commit/519a993f0408c8a10d2dfcc5d6f6cf403820fb3a', commit_message='Upload ResNetForImageClassification', commit_description='', oid='519a993f0408c8a10d2dfcc5d6f6cf403820fb3a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Arunavaonly/YouTube-Clickbait-Video-Detection-Model', endpoint='https://huggingface.co', repo_type='model', repo_id='Arunavaonly/YouTube-Clickbait-Video-Detection-Model'), pr_revision=None, pr_num=None)